<a href="https://colab.research.google.com/github/MarsZDF/argentum-v2/blob/master/argentum_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Argentum StateDiff Demo\n\nSimple state tracking for AI agents - see exactly how your agent's state evolves over time.

## Installation

In [ ]:
# Install from source (for demo)\n!pip install git+https://github.com/MarsZDF/argentum-v2.git\n\n# Or install from PyPI when published:\n# !pip install argentum

## Basic Usage

In [ ]:
from argentum import StateDiffTracker\n\n# Create tracker\ntracker = StateDiffTracker()\n\n# Track simple state changes\ntracker.capture(\"initial\", {\n    \"thoughts\": \"I need to help the user\",\n    \"plan\": [\"understand\", \"research\"],\n    \"status\": \"thinking\"\n})\n\ntracker.capture(\"updated\", {\n    \"thoughts\": \"I found the answer\", \n    \"plan\": [\"understand\", \"research\", \"respond\"],\n    \"status\": \"ready\",\n    \"answer\": \"The solution is X\"\n})\n\n# See what changed\ntracker.show_diff(\"initial\", \"updated\")

## Try It Yourself!\n\nCreate your own agent state tracking below:

In [ ]:
# Your playground - try tracking your own agent states!\nmy_tracker = StateDiffTracker()\n\n# TODO: Add your agent state examples here\n# Example:\n# my_tracker.capture(\"step1\", {\"your\": \"state\"})\n# my_tracker.capture(\"step2\", {\"your\": \"updated_state\"})\n# my_tracker.show_diff(\"step1\", \"step2\")